In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [3]:
pd.__version__

'1.0.4'

In [4]:
test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [5]:
test.shape

(200000, 5)

In [6]:
test.head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg
0,this club is really pretty and played good mus...,4.0,POSITIVE,POSITIVE,POSITIVE
1,Took my staff out for Lunch. We had a great t...,5.0,POSITIVE,POSITIVE,POSITIVE
2,"I like this place! Tasty food, delicious drink...",5.0,POSITIVE,POSITIVE,POSITIVE
3,Best chain ever for fresh squeezed juices and ...,5.0,POSITIVE,POSITIVE,POSITIVE
4,Amazing place!! Definitely super busy during w...,5.0,POSITIVE,POSITIVE,POSITIVE


In [7]:
from transformers import InputExample, InputFeatures, DistilBertTokenizerFast

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

In [9]:
from sklearn.utils import shuffle
test = shuffle(test)

In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [11]:
import time

In [12]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [33]:
def batch_encodings(rows):
    max_length = 256
    labels = rows['stars'].astype(int).tolist()

    batch_encoding = tokenizer.batch_encode_plus(
        rows['text'].tolist(), max_length=max_length, pad_to_max_length=True, return_token_type_ids=True
    )

    features = []
    for i in range(len(rows)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=str(labels[i]))
        features.append(feature)
        
    return features


In [34]:
test_sample = test.sample(10_000, random_state=42)

In [35]:
start = time.time()
features = batch_encodings(test_sample)
time.time() - start

3.097181797027588

In [36]:
features[0]

InputFeatures(input_ids=[101, 146, 1138, 1151, 1106, 1242, 19499, 12526, 1107, 1139, 1297, 113, 4062, 194, 1733, 114, 1105, 146, 1355, 1154, 16218, 1162, 7085, 26604, 1114, 1103, 1269, 11471, 119, 1337, 146, 1156, 1129, 1723, 1213, 1105, 185, 12831, 1174, 1105, 1243, 12049, 1114, 1103, 3813, 2546, 119, 1252, 117, 9371, 140, 14543, 117, 1184, 2171, 136, 146, 1108, 11196, 1118, 27688, 22600, 117, 1131, 2234, 1941, 1105, 3356, 1149, 1187, 1296, 2235, 1104, 1103, 7085, 26604, 140, 3190, 1413, 1127, 1388, 1105, 1173, 1131, 5534, 1228, 119, 4114, 1173, 146, 1108, 1176, 117, 145, 2591, 3048, 136, 11679, 2328, 2328, 1115, 112, 188, 1122, 136, 1337, 112, 188, 1165, 146, 1464, 1176, 21534, 117, 1142, 2762, 112, 189, 1177, 2213, 1303, 119, 146, 1455, 1106, 1267, 170, 2218, 140, 3190, 1580, 1115, 146, 1276, 3294, 1105, 2039, 1642, 1603, 119, 1153, 3716, 1917, 1107, 1103, 3686, 117, 3716, 1103, 3719, 1130, 1168, 10388, 3584, 117, 3716, 13178, 1183, 1105, 13080, 1105, 1238, 112, 189, 1517, 1294, 114

In [37]:
def gen():
    for ex in features:
        yield (
            {
                "input_ids": ex.input_ids,
                "attention_mask": ex.attention_mask,
                "token_type_ids": ex.token_type_ids,
            },
            ex.label,
        )

In [38]:
tf_dataset = tf.data.Dataset.from_generator(
    gen,
    ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.string),
    (
        {
            "input_ids": tf.TensorShape([None]),
            "attention_mask": tf.TensorShape([None]),
            "token_type_ids": tf.TensorShape([None]),
        },
        tf.TensorShape([]),
    ),
)

In [40]:
for example in tf_dataset.take(5):
    print(example[0])

{'input_ids': <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([  101,   146,  1138,  1151,  1106,  1242, 19499, 12526,  1107,
        1139,  1297,   113,  4062,   194,  1733,   114,  1105,   146,
        1355,  1154, 16218,  1162,  7085, 26604,  1114,  1103,  1269,
       11471,   119,  1337,   146,  1156,  1129,  1723,  1213,  1105,
         185, 12831,  1174,  1105,  1243, 12049,  1114,  1103,  3813,
        2546,   119,  1252,   117,  9371,   140, 14543,   117,  1184,
        2171,   136,   146,  1108, 11196,  1118, 27688, 22600,   117,
        1131,  2234,  1941,  1105,  3356,  1149,  1187,  1296,  2235,
        1104,  1103,  7085, 26604,   140,  3190,  1413,  1127,  1388,
        1105,  1173,  1131,  5534,  1228,   119,  4114,  1173,   146,
        1108,  1176,   117,   145,  2591,  3048,   136, 11679,  2328,
        2328,  1115,   112,   188,  1122,   136,  1337,   112,   188,
        1165,   146,  1464,  1176, 21534,   117,  1142,  2762,   112,
         189,  1177,  2213,  1

In [ ]:
# Write the `tf.Example` observations to the file.
with tf.io.TFRecordWriter(filename) as writer:
    for i in range(n_observations):
        example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
        writer.write(example)
